In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [79]:
base_url = "https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/"

In [80]:
response = requests.get(base_url)

In [4]:
response.status_code

200

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [81]:
products = soup.select(".product")
product = products[2]

In [62]:
name = product.select_one(".wd-entities-title a").text
prices = product.select(".woocommerce-Price-amount bdi")
regular_price = prices[0].text.replace('\xa0', '').replace("ден", "")
discount_price = regular_price
if(len(prices) == 2):
    discount_price = prices[1].text.replace('\xa0', '').replace("ден", "")
    
url_product_page = product.select_one(".wd-entities-title > a").attrs["href"]
url_add_to_cart = product.select_one(".wd-add-btn > a").attrs["href"]  


In [91]:
def extract_product_info(product, url):
    name = product.select_one(".wd-entities-title a").text
    prices = product.select(".woocommerce-Price-amount bdi")
    regular_price = (prices[0].text.replace('\xa0', '')
                     .replace("ден", "")
                     .replace(".", ""))    
    discount_price = 0
    if(len(prices) == 2):
        discount_price = (prices[1].text.replace('\xa0', '')
                          .replace("ден", "")
                          .replace(".", ""))        
    
    url_product_page = product.select_one(".wd-entities-title > a").attrs["href"]
    url_add_to_cart = url +""+ product.select_one(".wd-add-btn > a").attrs["href"]
    
    product_dict = {
        "ProductName": name,
        "RegularPrice": regular_price,
        "DiscountPrice": discount_price,
        "UrlProductPage": url_product_page,
        "UrlAddToCart": url_add_to_cart        
    }
    
    return product_dict

In [78]:
product_pages = soup.select(".wd-pagination .page-numbers li")
product_pages = product_pages[len(product_pages)-2].select_one("a").text
product_pages

'14'

In [97]:
all_products = []
for i in range(1, int(product_pages)+1):
    url = base_url + "page/" + str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    products = soup.select(".product")
    for product in products:
        result = extract_product_info(product, url)
        all_products.append(result)

In [101]:
all_products_df = pd.DataFrame(all_products)
all_products_df

,ProductName,RegularPrice,DiscountPrice,UrlProductPage,UrlAddToCart
0,Acer A315-23-A7KD,17590,0,https://clevershop.mk/product/acer-a315-23-a7kd/,https://clevershop.mk/product-category/mobilni...
1,Acer A315-23-R5P2,27490,0,https://clevershop.mk/product/acer-a315-23-r5p2/,https://clevershop.mk/product-category/mobilni...
2,ACER Aspire 1 A115-22,18999,15999,https://clevershop.mk/product/acer-aspire-1-nx...,https://clevershop.mk/product-category/mobilni...
3,Acer Aspire 3 A315-23-R26A,29990,0,https://clevershop.mk/product/acer-aspire-3-a3...,https://clevershop.mk/product-category/mobilni...
4,Acer Aspire 3 A315-58-33WK,24490,0,https://clevershop.mk/product/21498/,https://clevershop.mk/product-category/mobilni...
...,...,...,...,...,...
315,Monitor 27 Philips 272E1GAJ/00 VA 1ms 144Hz,12890,0,https://clevershop.mk/product/monitor-27-phili...,https://clevershop.mk/product-category/mobilni...
316,Philips 24″ 243V7QDSB,8390,0,https://clevershop.mk/product/philips-24%e2%80...,https://clevershop.mk/product-category/mobilni...
317,Philips 27″ 278E1A/00 4K UHD IPS,18990,0,https://clevershop.mk/product/hp-27%e2%80%b3-2...,https://clevershop.mk/product-category/mobilni...
318,Philips 279C9-00 MON LED 27″ 3840 x 2160 5Ms 6...,26990,0,https://clevershop.mk/product/philips-279c9-00...,https://clevershop.mk/product-category/mobilni...


In [105]:
all_products_df.to_csv("./data/clevershop_products.csv", index=False)

In [106]:
all_products_df.isnull().sum()

ProductName       0
RegularPrice      0
DiscountPrice     0
UrlProductPage    0
UrlAddToCart      0
dtype: int64

NameError: name 'pd' is not defined